In [2]:
import cv2
import numpy as np
import os
import pandas as pd

In [32]:
# Compute Bhattacharyya similarity (using histograms)
def compute_bhattacharyya(image1, image2):
    if len(image1.shape) == 3: #assumeixo que image1 i image2 tenen mateixa shape
        channels = image1.shape[-1]
    else:
        channels = 1


    hist1 = []
    hist2 = []
    # Get histograms
    for c in range(channels):
        hist1.append(cv2.calcHist([image1], [c], None, [256], (0,256)))
        hist2.append(cv2.calcHist([image2], [c], None, [256], (0,256)))

    # Compute Bhattacharyya distance between histograms
    bh_dist = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)

    # Return the Bhattacharyya similarity (1 - distance)
    return 1 - bh_dist

In [16]:
# Function to compute RMSE similarity between two images (directly on pixels)
def compute_rmse(image1, image2):
    # Convert images to float32 for precision
    image1 = image1.astype(np.float32)
    image2 = image2.astype(np.float32)
    
    # Compute RMSE
    mse = np.mean((image1 - image2) ** 2)
    rmse = np.sqrt(mse)
    
    # Return RMSE as a similarity measure
    return rmse

In [17]:
def compare_images(image1, image2, method='bhattacharyya'):
    if method == 'bhattacharyya':
        return compute_bhattacharyya(image1, image2)
    elif method == 'rmse':
        return compute_rmse(image1, image2)
    else:
        raise ValueError(f"Unknown method: {method}. Choose 'bhattacharyya' or 'rmse'.")

In [18]:
# Directories containing the noisy images and the ground truth images
noisy_images_dir = '/Users/abriil/Uni/master/C1/data/qsd1_w3'
gt_images_dir = '/Users/abriil/Uni/master/C1/data/qsd1_w3/non_augmented'

# List to store results for each image
results = []

# Process all images in the folder
comparison_method = 'rmse'  # Choose 'bhattacharyya' or 'rmse'

for img_name in os.listdir(noisy_images_dir):
    if img_name.endswith('.jpg') or img_name.endswith('.png'):
        # Load noisy image and corresponding ground truth image
        noisy_image_path = os.path.join(noisy_images_dir, img_name)
        gt_image_path = os.path.join(gt_images_dir, img_name)

        noisy_image = cv2.imread(noisy_image_path)
        gt_image = cv2.imread(gt_image_path)

        # Ensure both images are loaded correctly
        if noisy_image is None or gt_image is None:
            print(f"Error loading {img_name}")
            continue

        #Calculate similarity before filtering
        similarity_before = compare_images(noisy_image, gt_image, method=comparison_method)

        #Try parameters for each filter

        # Gaussian Blur - Different kernel sizes
        gb_sims = []
        for k in [(3, 3), (5, 5), (7, 7)]:
            gb_img = cv2.GaussianBlur(noisy_image, k, 0)
            gb_sims.append((k, compare_images(gb_img, gt_image, method=comparison_method)))
        
        # Median Blur - Different kernel sizes
        mb_sims = []
        for k in [3, 5, 7]:
            mb_img = cv2.medianBlur(noisy_image, k)
            mb_sims.append((k, compare_images(mb_img, gt_image, method=comparison_method)))
        
        # Bilateral Filter - Different d, sigmaColor, sigmaSpace
        bb_sims = []
        for (d, sigma_color, sigma_space) in [(9, 75, 75), (9, 100, 100), (9, 150, 150)]:
            bb_img = cv2.bilateralFilter(noisy_image, d, sigma_color, sigma_space)
            bb_sims.append(((d, sigma_color, sigma_space), compare_images(bb_img, gt_image, method=comparison_method)))

        # Non-Local Means (NLM) - Different h parameters
        nlm_sims = []
        for (h_luminance, h_color) in [(10, 10), (7, 21), (15, 15)]:
            nlm_img = cv2.fastNlMeansDenoisingColored(noisy_image, None, h_luminance, h_color, 7, 21)
            nlm_sims.append(((h_luminance, h_color), compare_images(nlm_img, gt_image, method=comparison_method)))

        # Find the best parameters for each filter
        best_gb_sim = max(gb_sims, key=lambda x: x[1]) if comparison_method == 'bhattacharyya' else min(gb_sims, key=lambda x: x[1])
        best_mb_sim = max(mb_sims, key=lambda x: x[1]) if comparison_method == 'bhattacharyya' else min(mb_sims, key=lambda x: x[1])
        best_bb_sim = max(bb_sims, key=lambda x: x[1]) if comparison_method == 'bhattacharyya' else min(bb_sims, key=lambda x: x[1])
        best_nlm_sim = max(nlm_sims, key=lambda x: x[1]) if comparison_method == 'bhattacharyya' else min(nlm_sims, key=lambda x: x[1])

        # Store the results for this image
        results.append({
            'Image': img_name,
            'Similarity (Before)': similarity_before,
            'Best Gaussian Blur (Kernel)': best_gb_sim[0],
            'Gaussian Blur (Similarity)': best_gb_sim[1],
            'Best Median Blur (Kernel)': best_mb_sim[0],
            'Median Blur (Similarity)': best_mb_sim[1],
            'Best Bilateral Filter (Params)': best_bb_sim[0],
            'Bilateral Filter (Similarity)': best_bb_sim[1],
            'Best NLM (Params)': best_nlm_sim[0],
            'NLM (Similarity)': best_nlm_sim[1]
        })

In [19]:
# Convert results to a pandas DataFrame for better organization and analysis
df_results_rmse = pd.DataFrame(results)

# Optionally, save the results to a CSV file
df_results_rmse.to_csv(f'comparison_{comparison_method}_results.csv', index=False)

In [20]:
df_results_rmse

,Image,Similarity (Before),Best Gaussian Blur (Kernel),Gaussian Blur (Similarity),Best Median Blur (Kernel),Median Blur (Similarity),Best Bilateral Filter (Params),Bilateral Filter (Similarity),Best NLM (Params),NLM (Similarity)
0,00025.png,0.000000,"(3, 3)",1.893584,3,1.616509,"(9, 75, 75)",0.813629,"(7, 21)",2.690032
1,00019.png,0.000000,"(3, 3)",4.757648,3,1.909239,"(9, 75, 75)",1.142973,"(7, 21)",2.936453
2,00025.jpg,1.508824,"(3, 3)",2.820085,3,2.653217,"(9, 75, 75)",4.394078,"(7, 21)",5.428494
3,00019.jpg,2.722002,"(3, 3)",3.401099,3,3.448440,"(9, 75, 75)",4.628955,"(7, 21)",4.783247
4,00018.jpg,15.067229,"(7, 7)",5.196736,7,3.681028,"(9, 150, 150)",5.004509,"(15, 15)",4.308164
5,00024.jpg,0.000000,"(3, 3)",9.094664,3,9.039985,"(9, 75, 75)",5.214107,"(7, 21)",5.201924
6,00018.png,0.000000,"(3, 3)",3.057924,3,0.835144,"(9, 75, 75)",0.712178,"(7, 21)",1.796338
7,00024.png,0.000000,"(3, 3)",6.405565,3,0.771350,"(9, 75, 75)",0.973778,"(7, 21)",2.204138
8,00026.png,0.000000,"(3, 3)",3.116697,3,0.639781,"(9, 75, 75)",0.626693,"(7, 21)",1.657973
9,00026.jpg,23.588249,"(7, 7)",23.410727,5,23.503849,"(9, 75, 75)",23.313351,"(15, 15)",23.405584


In [4]:
import math

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float("inf")
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [10]:
def ssim(img1, img2):
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1 ** 2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2 ** 2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / (
        (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    )
    return ssim_map.mean()

def calculate_ssim(img1, img2):
    """calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    """
    if not img1.shape == img2.shape:
        raise ValueError("Input images must have the same dimensions.")
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError("Wrong input image dimensions.")


In [30]:
import piq
import lpips
import torch
import numpy as np

# Function to calculate LPIPS metric between two images
def calculate_lpips(image1, image2, net='alex'):
    """
    Compute the LPIPS metric between two images.
    
    Parameters:
    - image1: First image as a NumPy array (H x W x C), where C is 3 for RGB.
    - image2: Second image as a NumPy array (H x W x C), where C is 3 for RGB.
    - net: The network architecture used for LPIPS computation ('alex', 'vgg', 'squeeze'). Default is 'alex'.
    
    Returns:
    - lpips_distance: The LPIPS distance between image1 and image2.
    """
    # Convert images from NumPy to PyTorch Tensors and normalize to [-1, 1]
    # (LPIPS expects inputs in the range [-1, 1] and in the format (N, C, H, W))
    image1_torch = torch.from_numpy(image1).permute(2, 0, 1).unsqueeze(0).float() / 127.5 - 1.0
    image2_torch = torch.from_numpy(image2).permute(2, 0, 1).unsqueeze(0).float() / 127.5 - 1.0

    # Initialize the LPIPS model (choose the backbone: 'alex', 'vgg', 'squeeze')
    loss_fn = lpips.LPIPS(net=net)

    # Calculate LPIPS distance
    lpips_distance = loss_fn(image1_torch, image2_torch)

    return lpips_distance.item()



In [32]:
image1 = cv2.imread('/Users/abriil/Uni/master/C1/data/qsd1_w3/00005.jpg')
image2 = cv2.imread('/Users/abriil/Uni/master/C1/data/qsd1_w3/non_augmented/00005.jpg')

calculate_psnr(image1, image2), calculate_ssim(image1, image2), calculate_lpips(image1, image2) 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/anaconda3/envs/prova/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


(24.29165523734892, 0.5735294129751751, 0.6599388122558594)

In [35]:
#comprovació de que tinguin sentit tenint en compte: PSNR com més alt millor, SSIM com més aprop d 1 millor, LPIPS com més aprop d 0 millor
calculate_psnr(image1, image1), calculate_ssim(image1, image1), calculate_lpips(image1, image1)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/anaconda3/envs/prova/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


(inf, 1.0, 0.0)